In [3]:
import ee
ee.Authenticate()
ee.Initialize()


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=IxsDx9WmR8gVDAapdNYiCjVGrUjGccf7O5X224HUo_g&tc=WHDMtOFZkOKSeog6mt3_55SK_3nmk8xBSmQEIhcO6U4&cc=Tj9Ruvn58g5g9SzwxNo5sGmoCEBQFRaS8WcM1eZmWko

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhWYitTB-FOXNbTlEN4eqid_myrIvnyUxZ_MY6gfDf7bzeDsCyQ2VXk

Successfully saved authorization token.


In [4]:
import pandas as pd
import numpy as np
import matplotlib
import scipy
import folium
import shapely
from IPython.display import Image

In [5]:
!pip install geopandas rasterio pygeohydro geemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 112.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.1/221.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 114.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import geopandas as gpd
import rasterio
import pygeohydro

In [7]:
%cd drive/MyDrive/Colab Notebooks/meadows/

/content/drive/MyDrive/Colab Notebooks/meadows


In [8]:
meadows_df = gpd.read_parquet("./meadows_data/ucdmeadows/ucdmeadows.parquet")
meadows_df.head()
meadows_df = meadows_df.to_crs("WGS84")

In [9]:
## get all the data and layers that intersect with points in one huc12
sample = meadows_df.loc[meadows_df['HUC12'] == '180300020601']
sample

,geometry,AREA_ACRE,STATE,ID,HUC12,OWNERSHIP,EDGE_COMPLEXITY,DOM_ROCKTYPE,VEG_MAJORITY,COKEY,...,uniqueID,MDW_DEM_SLOPE,ED_MIN_FStopo_Trails,ED_MIN_FStopo_Roads,POUR_POINT_LAT,POUR_POINT_LON,StreamSlopeGrade,MDW_CATCHMENT_AREA,SHAPE_Length,SHAPE_Area
0,"MULTIPOLYGON (((-118.33411 35.42631, -118.3340...",2.283849,CA,UCDSNM000005,180300020601,Private,1.368988,granodiorite,Conifer,13197316,...,1,12.157462,336.005951,593.969727,35.426085,-118.334596,0.156945,6.283005e+05,466.548680,9242.407193
1,"MULTIPOLYGON (((-118.32744 35.44003, -118.3276...",17.017459,CA,UCDSNM000007,180300020601,Private,1.303125,granodiorite,Conifer,13197209,...,2,3.148981,488.773987,10.000000,35.443429,-118.326901,0.042227,4.897827e+06,1212.264166,68867.214181
2,"MULTIPOLYGON (((-118.29222 35.44776, -118.2921...",153.904101,CA,UCDSNM000008,180300020601,Private,2.944442,granodiorite,Conifer,13197209,...,3,2.654701,0.000000,0.000000,35.444935,-118.316133,0.020900,1.284576e+07,8237.434596,622827.799550
3,"MULTIPOLYGON (((-118.31680 35.45400, -118.3167...",63.880979,CA,UCDSNM000010,180300020601,Sequoia National Forest,2.027670,granodiorite,Conifer,13197209,...,4,4.535124,0.000000,0.000000,35.454702,-118.321363,0.012879,3.845936e+07,3654.659953,258517.150924
6,"MULTIPOLYGON (((-118.33593 35.45644, -118.3358...",54.327739,CA,UCDSNM000015,180300020601,Private,3.567054,granodiorite,Conifer,13197209,...,7,5.618661,0.000000,0.000000,35.452504,-118.330456,0.054110,8.313676e+06,5929.038960,219856.558975
9,"MULTIPOLYGON (((-118.33598 35.48463, -118.3359...",14.153260,CA,UCDSNM000018,180300020601,Sequoia National Forest,1.869578,granodiorite,Conifer,13197316,...,10,9.615345,107.703293,0.000000,35.484091,-118.335838,0.133437,2.144075e+06,1586.118374,57276.211293
11,"MULTIPOLYGON (((-118.34457 35.50743, -118.3444...",9.075996,CA,UCDSNM000021,180300020601,Sequoia National Forest,1.222355,granodiorite,Conifer,13197316,...,12,5.487450,0.000000,0.000000,NaN,NaN,NaN,3.168226e+05,830.440239,36729.252923


In [10]:
wbd = pygeohydro.watershed.WBD('huc12')
data = wbd.byids(field='huc12', fids=['180201210605'])
data = data.to_crs(meadows_df.crs)
data.iloc[0].geometry
data.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich


1.    get landsat that intersects with data geometry where data is WBD return union month=month
2.   get monthly average for all bands in month mosaiced
3.   add bands for elevation, chili, mTPI, slope
4.   make feature collection from meadows
5.   split into train/test
6.   train classifier
7.   get metrics

In [11]:
import geemap.foliumap as geemap

In [12]:
from geemap.plot import geopandas_to_ee
study_area = geopandas_to_ee(data)

In [13]:
def bitwise_extract(input, start=3, end=4):
        mask_size = ee.Number(1).add(end).subtract(start)
        mask = ee.Number(1).leftShift(mask_size).subtract(1)
        return input.rightShift(start).bitwiseAnd(mask)

def mask_clouds(image):
    # https://spatialthoughts.com/2021/08/19/qa-bands-bitmasks-gee/
    # https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#bands

    qa = image.select('QA_PIXEL')
    # clouds are Bit 3, cloud shadow is Bit 4
    cloud_mask = bitwise_extract(qa).eq(0)
    image = image.updateMask(cloud_mask)
    return image

def scale_factor(image):
    optical = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    image = image.addBands(optical, None, True)
    image = image.addBands(thermal, None, True)
    return image

def compute_indices(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename("NDVI")
    image = image.addBands(ndvi)
    return image

def clip_sa(ic, study_area):
    def clip(image):
      return image.clip(study_area)
    return ic.map(clip)

ls =  ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate('2013-05-01', '2013-09-30')
ls = ls.filterBounds(study_area)
ls = clip_sa(ls, study_area)
ls = ls.map(scale_factor)
ls = ls.map(mask_clouds)
ls = ls.map(compute_indices)

In [14]:
Map = geemap.Map(zoom=4, crs="EPSG4326")
Map.set_center(-150.7, 21.7)
first = ls.first()
vis_params = {'bands':['SR_B4', 'SR_B3', 'SR_B2'], 'min':0, 'max':0.2}
ndvi_vis_params = {'bands': ['NDVI'], 'min':-1, 'max':1}
Map.addLayer(first, vis_params, 'landsat')
Map.addLayer(first, ndvi_vis_params, 'ndvi')
Map

In [15]:
Map = geemap.Map(zoom=4, crs="EPSG4326")
Map.set_center(-150.7, 21.7)
land_cover = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')
land_cover = land_cover.filter(ee.Filter.eq('system:index', '2019')).first()
land_cover = land_cover.select('landcover')
Map.addLayer(land_cover, None, 'Landcover')
Map

In [16]:
Map = geemap.Map(zoom=4, crs="EPSG4326")
Map.set_center(-150.7, 21.7)
dem = ee.Image('USGS/SRTMGL1_003');
elevation = dem.select('elevation');
slope = ee.Terrain.slope(elevation);
Map.addLayer(slope, {'min': 0, 'max': 60}, 'slope');
Map.addLayer(elevation, {'min': 0, 'max':3000}, 'elevation');
# add flow accumulation
# add TPI
# add chili
# add aspect
# add curvature
Map

In [ ]:
# tpi

In [19]:
Map = geemap.Map(zoom=4, crs="EPSG4326")
Map.set_center(-150.7, 21.7)
naip = ee.ImageCollection('USDA/NAIP/DOQQ')
naip = naip.filter(ee.Filter.date('2019-06-01', '2020-07-01'))
truecolor = naip.select(['R', 'G', 'B']);
vis_params = {
  min: 0,
  max: 255,
};
Map.addLayer(truecolor, vis_params, 'True Color');
Map

In [98]:
wbd = pygeohydro.watershed.WBD('huc12')
data = wbd.byids(field='huc12', fids=['180400090505'])
#meadows_df['huc10'] = meadows_df['HUC12'].apply(lambda x: x[:-2])
meadows = meadows_df.loc[meadows_df['HUC12'].isin(['180400090505'])]
study_area_ee = geopandas_to_ee(data)
meadows_ee = geopandas_to_ee(meadows)

In [104]:
# Option 1:
# given a set of points (some meadow, some non-meadow)
# use Image.sampleRegions on the points to get the feature values
# train cart classifier

Map = geemap.Map(zoom=4, crs="EPSG4326")
Map.set_center(-149.9, 20.5)
naip = ee.ImageCollection('USDA/NAIP/DOQQ')
naip = naip.filter(ee.Filter.date('2018-05-01', '2020-07-01'))
naip = naip.filterBounds(study_area.geometry())
naip = naip.median()
naip = naip.clip(study_area_ee.geometry())
truecolor = naip.select(['R', 'G', 'B']);
vis_params = {
  'min': 0,
  'max': 255,
  'gamma': [1.4, 1.4, 1.4]
};
Map.addLayer(truecolor, vis_params, 'True Color')
Map.addLayer(meadows_ee, {'opacity': 0.5, 'color': 'green'}, 'meadows')
Map

In [96]:
meadows

,geometry,AREA_ACRE,STATE,ID,HUC12,OWNERSHIP,EDGE_COMPLEXITY,DOM_ROCKTYPE,VEG_MAJORITY,COKEY,...,MDW_DEM_SLOPE,ED_MIN_FStopo_Trails,ED_MIN_FStopo_Roads,POUR_POINT_LAT,POUR_POINT_LON,StreamSlopeGrade,MDW_CATCHMENT_AREA,SHAPE_Length,SHAPE_Area,huc10
6515,"MULTIPOLYGON (((-119.93051 37.87948, -119.9305...",6.196644,CA,UCDSNM009471,180400090505,Stanislaus National Forest,2.086228,granodiorite,Hardwood,13180872,...,12.060785,0.000000,956.347229,37.879795,-119.928562,0.120018,4.663950e+05,1171.126757,25076.927367,1804000905
6516,"MULTIPOLYGON (((-119.84904 37.87915, -119.8493...",12.341938,CA,UCDSNM009476,180400090505,Private,1.463192,granodiorite,Conifer,13180872,...,2.798101,422.018951,0.000000,37.880828,-119.848151,0.044701,1.232260e+06,1159.195839,49946.051299,1804000905
6517,"MULTIPOLYGON (((-119.85847 37.88350, -119.8584...",6.437789,CA,UCDSNM009496,180400090505,Private,1.217274,glacial drift,Riparian,13180485,...,2.987360,623.939087,20.000000,37.881959,-119.859947,0.011743,3.863020e+06,696.499184,26052.807596,1804000905
6519,"MULTIPOLYGON (((-119.84818 37.88251, -119.8478...",31.900228,CA,UCDSNM009521,180400090505,Private,1.937796,glacial drift,Riparian,13180872,...,1.696487,0.000000,0.000000,37.882401,-119.853980,0.019303,3.477523e+06,2468.134726,129095.643757,1804000905
6520,"MULTIPOLYGON (((-119.89384 37.88554, -119.8938...",2.719405,CA,UCDSNM009522,180400090505,Stanislaus National Forest,2.464217,granodiorite,Shrubland,13180872,...,8.637568,0.000000,1056.834839,37.885730,-119.894981,0.104522,6.107071e+05,916.388642,11005.042281,1804000905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18276,"MULTIPOLYGON (((-119.59125 38.17346, -119.5912...",6.472451,CA,UCDSNM011158,180400090503,Yosemite National Park,2.545644,granodiorite,Conifer,13211333,...,3.108184,161.245148,10847.400391,38.171368,-119.591070,-0.015172,1.390562e+06,1460.480846,26193.078048,1804000905
18277,"MULTIPOLYGON (((-119.59049 38.17462, -119.5904...",5.179629,CA,UCDSNM011164,180400090503,Yosemite National Park,2.168017,water,Conifer,14185300,...,8.734879,224.722046,10813.995117,NaN,NaN,NaN,6.554907e+04,1112.694171,20961.213695,1804000905
18280,"MULTIPOLYGON (((-119.59055 38.17746, -119.5906...",4.416502,CA,UCDSNM011183,180400090503,Yosemite National Park,1.989581,water,Conifer,14185300,...,6.865256,0.000000,10469.459961,NaN,NaN,NaN,4.694937e+05,942.897676,17872.948086,1804000905
18281,"MULTIPOLYGON (((-119.58458 38.17923, -119.5845...",11.206189,CA,UCDSNM011185,180400090503,Yosemite National Park,2.111341,water,Conifer,13211333,...,5.404061,0.000000,10438.970703,NaN,NaN,NaN,5.116326e+05,1593.864035,45349.837102,1804000905
